In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data = pd.read_csv("/home/eiji/project/g/preprocessed_data/Fridge_LE-SS.csv")
data

,hour,minute,second,fridge_temperature,temp_condition_encoded,label
0,0.418197,0.647441,1.311360,1.642293,1,0
1,0.418197,0.647441,1.369743,0.414644,1,0
2,0.418197,0.647441,1.428126,-1.419933,0,0
3,0.418197,0.647441,1.486508,-0.647480,0,0
4,0.418197,0.647441,1.544891,0.980190,1,0
...,...,...,...,...,...,...
587071,0.418197,0.948827,-0.732031,-0.868180,0,0
587072,0.418197,0.948827,-0.732031,-0.302634,1,0
587073,0.418197,0.948827,-0.732031,1.559530,1,0
587074,0.418197,0.948827,-0.673649,-0.178490,1,0


In [2]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2024-08-22 13:19:19.950655: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-22 13:19:19.976251: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 13:19:20.373672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-22 13:19:20.656654: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

In [4]:
X = data[['hour', 'minute', 'second', 'fridge_temperature', 'temp_condition_encoded']].values
y = data['label'].values

In [5]:
window_size = 30
X_windows = []
y_windows = []

for i in range(len(X) - window_size + 1):
    X_windows.append(X[i:i + window_size])
    y_windows.append(y[i + window_size - 1])  # ラベルはウィンドウの最後のデータに対応させます

X_windows = np.array(X_windows)
y_windows = np.array(y_windows)


# トレーニングデータとテストデータに分割
X_train, X_temp, y_train, y_temp = train_test_split(X_windows, y_windows, test_size=0.3, random_state=42, stratify=y_windows)
# テストデータをさらに検証データとテストデータに分割
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# LSTMモデルの構築
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# モデルのコンパイル
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# モデルの概要
model.summary()

/home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,251 (43.95 KB)

 Trainable params: 11,251 (43.95 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# モデルの訓練
# モデルの訓練
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)


Epoch 1/10


I0000 00:00:1724300403.025408   25018 service.cc:145] XLA service 0x7c1a6000a790 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724300403.025429   25018 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-08-22 13:20:03.045720: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


  45/6421 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.6904 - loss: 0.5964    

I0000 00:00:1724300403.344786   25018 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6421/6421 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.8949 - loss: 0.2588 - val_accuracy: 0.9613 - val_loss: 0.1065
Epoch 2/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9624 - loss: 0.1043 - val_accuracy: 0.9721 - val_loss: 0.0811
Epoch 3/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.9693 - loss: 0.0851 - val_accuracy: 0.9756 - val_loss: 0.0690
Epoch 4/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.9719 - loss: 0.0780 - val_accuracy: 0.9759 - val_loss: 0.0663
Epoch 5/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.9759 - loss: 0.0669 - val_accuracy: 0.9777 - val_loss: 0.0610
Epoch 6/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.9782 - loss: 0.0612 - val_accuracy: 0.9803 - val_loss: 0.0565
Epoch 7/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.9789 - loss: 0.0584 - val_accuracy: 0.9807 - val_loss: 0.0547
Epoch 8/10
6421/6421 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.9799 - loss: 0.0551 - val

In [8]:
# モデルの評価
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {accuracy:.4f}')

2752/2752 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9834 - loss: 0.0487
Test Accuracy: 0.9829


2752/2752 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


precision    recall  f1-score   support

           0       0.99      0.99      0.99     75120
           1       0.95      0.93      0.94     12938

    accuracy                           0.98     88058
   macro avg       0.97      0.96      0.97     88058
weighted avg       0.98      0.98      0.98     88058



In [9]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 予測の実行
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # 閾値0.5でクラスに変換

# 混同行列の計算
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# 混同行列の表示
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# 精度、適合率、再現率、F1スコアのレポート
print(classification_report(y_test, y_pred_classes))


In [ ]:
import tensorflow as tf

# TensorFlowのバージョンを確認
print("TensorFlow version:", tf.__version__)

# 利用可能なGPUリストを表示
print("Available GPU devices:", tf.config.list_physical_devices('GPU'))
